In [59]:
import numpy as np

### Data

In [60]:
import sklearn
from pgmpy.utils import get_example_model

In [129]:
dataset_name = "insurance"

In [130]:
ground_bn = get_example_model(dataset_name)

dataset = ground_bn.simulate(n_samples = 5000)

  0%|          | 0/27 [00:00<?, ?it/s]

In [131]:
dataset.columns

Index(['SeniorTrain', 'HomeBase', 'Airbag', 'Theft', 'ILiCost', 'RiskAversion',
       'Antilock', 'DrivingSkill', 'OtherCar', 'GoodStudent', 'PropCost',
       'ThisCarCost', 'MedCost', 'ThisCarDam', 'Accident', 'DrivHist',
       'SocioEcon', 'Cushioning', 'DrivQuality', 'Mileage', 'VehicleYear',
       'CarValue', 'Age', 'AntiTheft', 'OtherCarCost', 'MakeModel',
       'RuggedAuto'],
      dtype='object')

In [132]:
target_name = 'Accident'
feature_names = list(dataset.drop(target_name, axis = 1).columns)
variables = list(dataset.columns)

In [133]:
X = dataset.drop(target_name, axis = 1).to_numpy()
y = dataset[target_name].to_numpy()

In [134]:
feature_encoder = sklearn.preprocessing.OrdinalEncoder()
feature_encoder.fit(X)

target_encoder = sklearn.preprocessing.LabelEncoder()
target_encoder.fit(y.squeeze())

X = feature_encoder.transform(X)
y = target_encoder.transform(y)

In [135]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y.squeeze(), test_size = 0.2, random_state = 51)

### Model

In [136]:
from tensorflow import keras

In [137]:
def build_model(input_shape, classes, n_layers = 5, n_nodes_l = 2, n_nodes_u = 10, optimizer = 'adam', learning_rate = 0.001):

    '''
    Dynamically creates fully-connected neural network given hyperparameters.
    '''
    
    model = keras.Sequential()
    model.add(keras.Input(shape = (input_shape,)))
    for i in range(n_layers):
        model.add(keras.layers.Dense(np.random.choice([i for i in range(n_nodes_l, n_nodes_u)]), activation = np.random.choice(['relu', 'sigmoid'])))
    model.add(keras.layers.Dense(classes, activation = 'softmax'))

    if optimizer == 'sgd':
        optim = keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'adam':
        optim = keras.optimizers.Adam(learning_rate = learning_rate)
    
    model.compile(optimizer = optim, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [138]:
n_classes = len(np.unique(y_train))
input_shape = len(X_train[0])
n_layers = 5
n_nodes_l = 5
n_nodes_u = 25
optimizer = 'adam'
learning_rate = 0.01
epochs = 25
batch_size = 100

In [139]:
model = build_model(
    input_shape,
    n_classes,
    n_layers = n_layers, 
    n_nodes_l = n_nodes_l,
    n_nodes_u = n_nodes_u, 
    optimizer = optimizer, 
    learning_rate = learning_rate
)

model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_split = 0.2)
model.summary()

Epoch 1/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5763 - loss: 0.9128 - val_accuracy: 0.7350 - val_loss: 0.6254
Epoch 2/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7581 - loss: 0.5282 - val_accuracy: 0.8913 - val_loss: 0.3796
Epoch 3/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8950 - loss: 0.3295 - val_accuracy: 0.9137 - val_loss: 0.2887
Epoch 4/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9212 - loss: 0.2553 - val_accuracy: 0.9250 - val_loss: 0.2416
Epoch 5/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9253 - loss: 0.2365 - val_accuracy: 0.9225 - val_loss: 0.2457
Epoch 6/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9331 - loss: 0.2203 - val_accuracy: 0.9287 - val_loss: 0.2352
Epoch 7/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9372 - loss: 0.1992 - val_accuracy: 0.9337 - val_loss: 0.2011
Epoch 8/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9403 - loss: 0.1882 - val_accuracy: 0.9337 - val_loss:

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_42 (Dense)                │ (None, 22)             │           594 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 7)              │           161 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 14)             │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 9)              │           135 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 5)              │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 4)              │            24 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,230 (12.62 KB)

 Trainable params: 1,076 (4.20 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,154 (8.42 KB)

In [140]:
y_pred = model(X_test)
y_pred = np.argmax(y_pred, axis = 1)

mlp_accuracy = sklearn.metrics.accuracy_score(y_pred, y_test)
print(f"Test Accuracy: {mlp_accuracy:.4f}")

Test Accuracy: 0.9520


#### Explanations

In [141]:
import os
import json
from modules.method import Method
from modules.metrics import markov_blanket, structure_metrics, distribution_drift, average_distribution_drift, divergence_plot, fidelity_plot

In [142]:
results = []
experiment_name = os.path.join("./data/results", dataset_name)
ground_mb = markov_blanket(ground_bn, target_name)
n_trials = 20

In [143]:
for _ in range(100):
    
    method = Method(
        model = model, 
        training_data = X_train, 
        feature_names = feature_names, 
        target_name = target_name, 
        n_samples = 5000, 
        rep_prob = 0.5
    )

    instance_idx = np.random.randint(X_test.shape[0])
    instance = X_test[instance_idx]
    
    method.log_data(instance)
    method.get_structure()

    precision, recall, f1, accuracy, confusion_matrix = structure_metrics(ground_mb, method.mb, variables)

    # Features to freeze while randomizing the rest
    no_features = [] # randomize all features (baseline)
    method_features = list(method.blanket) # freeze features returned from markov blanket of method
    ground_features = list(ground_mb.get_markov_blanket(target_name)) # freeze features from ground-truth markov blanket of target
    out_ground_features = [f for f in feature_names if f not in ground_features] # features outside of ground-truth markov blanket
    out_method_features = [f for f in feature_names if f not in method_features] # features outside of explainer markov blanket
    
    no_new, no_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = no_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = n_trials
    )
    
    method_new, method_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = method_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = n_trials
    )
    
    ground_new, ground_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = ground_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = n_trials
    )
    
    out_ground_new, out_ground_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = out_ground_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = n_trials
    )
    
    out_method_new, out_method_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = out_method_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = n_trials
    )
    
    no_avg, no_values = average_distribution_drift(no_og, no_new)
    method_avg, method_values = average_distribution_drift(method_og, method_new)
    ground_avg, ground_values = average_distribution_drift(ground_og, ground_new)
    out_ground_avg, out_ground_values = average_distribution_drift(out_ground_og, out_ground_new)
    out_method_avg, out_method_values = average_distribution_drift(out_method_og, out_method_new)
    
    js_values = np.stack((no_values, method_values, ground_values, out_ground_values, out_method_values))
    method_names = ['No Features', 'Method MB Features', 'Ground MB Features', 'Outside Ground MB Features', 'Outside Method MB Features']

    print(f'= = = = = = = = = = MARKOV BLANKET RETRIEVAL RESULTS = = = = = = = = = =')
    print(f'Precision: {precision} | Recall: {recall} | F1: {f1} | Accuracy: {accuracy}')
    print(f'Confusion Matrix: \n{confusion_matrix}\n')
    
    print(f'= = = = = = = = = = FIDELITY TO MODEL = = = = = = = = = =')
    print(f'No Features Divergence: {no_avg}')
    print(f'Method MB Features Divergence: {method_avg}')
    print(f'Ground MB Features Divergence: {ground_avg}')
    print(f'Outside Ground MB Featues Divergence: {out_ground_avg}')
    print(f'Outside Method MB Features Divergence: {out_method_avg}')

    results.append({

        "dataset": dataset_name,

        "target": target_name,
        "features": feature_names,
        "mlp_accuracy": float(mlp_accuracy),
        
        "method_structure": list(method.mb.edges()),
        "ground_structure": list(ground_mb.edges()),

        "precision": float(precision), 
        "recall": float(recall), 
        "f1": float(f1), 
        "accuracy": float(accuracy), 
        
        "method_features": method_features,
        "ground_features": ground_features,
        "out_ground_features": out_ground_features,
        "out_method_features": out_method_features,
        
        "no_new": no_new.tolist(),
        "method_new": method_new.tolist(),
        "ground_new": ground_new.tolist(),
        "out_ground_new": out_ground_new.tolist(),
        "out_method_new": out_method_new.tolist(),
        
        "no_og": no_og.tolist(),
        "method_og": method_og.tolist(),
        "ground_og": ground_og.tolist(),
        "out_ground_og": out_ground_og.tolist(),
        "out_method_og": out_method_og.tolist(),
        
        "no_values": list(no_values),
        "method_values": list(method_values),
        "ground_values": list(ground_values),
        "out_ground_values": list(out_ground_values),
        "out_method_values": list(out_method_values),
        
        "no_avg": float(no_avg),
        "method_avg": float(method_avg),
        "ground_avg": float(ground_avg),
        "out_ground_avg": float(out_ground_avg),
        "out_method_avg": float(out_method_avg),
    })

    with open(f'{experiment_name}.json', 'w') as f:

        json.dump(results, f, indent = 4)
    '''
    divergence_plot(js_values, method_names)
    
    fidelity_plot(no_new, no_og, method_names[0])
    fidelity_plot(method_new, method_og, method_names[1])
    fidelity_plot(ground_new, ground_og, method_names[2])
    fidelity_plot(out_ground_new, out_ground_og, method_names[3])
    fidelity_plot(out_method_new, out_method_og, method_names[4])
    '''

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step


INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'SeniorTrain': 'C', 'HomeBase': 'C', 'Airbag': 'C', 'Theft': 'C', 'ILiCost': 'C', 'RiskAversion': 'C', 'Antilock': 'C', 'DrivingSkill': 'C', 'OtherCar': 'C', 'GoodStudent': 'C', 'PropCost': 'C', 'ThisCarCost': 'C', 'MedCost': 'C', 'ThisCarDam': 'C', 'DrivHist': 'C', 'SocioEcon': 'C', 'Cushioning': 'C', 'DrivQuality': 'C', 'Mileage': 'C', 'VehicleYear': 'C', 'CarValue': 'C', 'Age': 'C', 'AntiTheft': 'C', 'OtherCarCost': 'C', 'MakeModel': 'C', 'RuggedAuto': 'C', 'Accident': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'SeniorTrain': 'C', 'HomeBase': 'C', 'Airbag': 'C', 'Theft': 'C', 'ILiCost': 'C', 'RiskAversion': 'C', 'Antilock': 'C', 'DrivingSkill': 'C', 'OtherCar': 'C', 'GoodStudent': 'C', 'PropCost': 'C', 'ThisCarCost': 'C', 'MedCost': 'C', 'ThisCarDam': 'C', 'DrivHist': 'C', 'SocioEcon': 'C', 'Cushioning': 'C', 'D

  0%|          | 0/1000000 [00:00<?, ?it/s]

= = = = = = = = = = MARKOV BLANKET RETRIEVAL RESULTS = = = = = = = = = =
Precision: 0.2222222222222222 | Recall: 0.3333333333333333 | F1: 0.26666666666666666 | Accuracy: 0.8044444444444444
Confusion Matrix: 
[[173  28]
 [ 16   8]]

= = = = = = = = = = FIDELITY TO MODEL = = = = = = = = = =
No Features Divergence: 0.6721384708502885
Method MB Features Divergence: 0.00012470908328507724
Ground MB Features Divergence: 8.582416484322155e-05
Outside Ground MB Featues Divergence: 0.00026624178387854813
Outside Method MB Features Divergence: 0.10152184563758208
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step


INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'SeniorTrain': 'C', 'HomeBase': 'C', 'Airbag': 'C', 'Theft': 'C', 'ILiCost': 'C', 'RiskAversion': 'C', 'Antilock': 'C', 'DrivingSkill': 'C', 'OtherCar': 'C', 'GoodStudent': 'C', 'PropCost': 'C', 'ThisCarCost': 'C', 'MedCost': 'C', 'ThisCarDam': 'C', 'DrivHist': 'C', 'SocioEcon': 'C', 'Cushioning': 'C', 'DrivQuality': 'C', 'Mileage': 'C', 'VehicleYear': 'C', 'CarValue': 'C', 'Age': 'C', 'AntiTheft': 'C', 'OtherCarCost': 'C', 'MakeModel': 'C', 'RuggedAuto': 'C', 'Accident': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'SeniorTrain': 'C', 'HomeBase': 'C', 'Airbag': 'C', 'Theft': 'C', 'ILiCost': 'C', 'RiskAversion': 'C', 'Antilock': 'C', 'DrivingSkill': 'C', 'OtherCar': 'C', 'GoodStudent': 'C', 'PropCost': 'C', 'ThisCarCost': 'C', 'MedCost': 'C', 'ThisCarDam': 'C', 'DrivHist': 'C', 'SocioEcon': 'C', 'Cushioning': 'C', 'D

...

  0%|          | 0/1000000 [00:00<?, ?it/s]

= = = = = = = = = = MARKOV BLANKET RETRIEVAL RESULTS = = = = = = = = = =
Precision: 0.2777777777777778 | Recall: 0.4166666666666667 | F1: 0.3333333333333333 | Accuracy: 0.7959183673469388
Confusion Matrix: 
[[146  26]
 [ 14  10]]

= = = = = = = = = = FIDELITY TO MODEL = = = = = = = = = =
No Features Divergence: 0.673235058791253
Method MB Features Divergence: 0.000984394817291037
Ground MB Features Divergence: 0.0009853003617402432
Outside Ground MB Featues Divergence: 0.03441290246941409
Outside Method MB Features Divergence: 0.03473850737409657
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step


INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'SeniorTrain': 'C', 'HomeBase': 'C', 'Airbag': 'C', 'Theft': 'C', 'ILiCost': 'C', 'RiskAversion': 'C', 'Antilock': 'C', 'DrivingSkill': 'C', 'OtherCar': 'C', 'GoodStudent': 'C', 'PropCost': 'C', 'ThisCarCost': 'C', 'MedCost': 'C', 'ThisCarDam': 'C', 'DrivHist': 'C', 'SocioEcon': 'C', 'Cushioning': 'C', 'DrivQuality': 'C', 'Mileage': 'C', 'VehicleYear': 'C', 'CarValue': 'C', 'Age': 'C', 'AntiTheft': 'C', 'OtherCarCost': 'C', 'MakeModel': 'C', 'RuggedAuto': 'C', 'Accident': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'SeniorTrain': 'C', 'HomeBase': 'C', 'Airbag': 'C', 'Theft': 'C', 'ILiCost': 'C', 'RiskAversion': 'C', 'Antilock': 'C', 'DrivingSkill': 'C', 'OtherCar': 'C', 'GoodStudent': 'C', 'PropCost': 'C', 'ThisCarCost': 'C', 'MedCost': 'C', 'ThisCarDam': 'C', 'DrivHist': 'C', 'SocioEcon': 'C', 'Cushioning': 'C', 'D

  0%|          | 0/1000000 [00:00<?, ?it/s]

= = = = = = = = = = MARKOV BLANKET RETRIEVAL RESULTS = = = = = = = = = =
Precision: 0.36363636363636365 | Recall: 0.3333333333333333 | F1: 0.34782608695652173 | Accuracy: 0.8666666666666667
Confusion Matrix: 
[[187  14]
 [ 16   8]]

= = = = = = = = = = FIDELITY TO MODEL = = = = = = = = = =
No Features Divergence: 0.31218369497649456
Method MB Features Divergence: 0.00012463266531590822
Ground MB Features Divergence: 0.0005545721701226627
Outside Ground MB Featues Divergence: 0.18275026473390804
Outside Method MB Features Divergence: 0.4613232880006416
